In [ ]:
chunksize = 10 ** 6

In [ ]:
#insert data name.basic
with pd.read_csv("name.basics.tsv",sep='\t', chunksize=chunksize,error_bad_lines=False) as reader:
    for chunk in reader:
        name_basics=chunk   
        name_basics['birthYear']=name_basics['birthYear'].replace(r'\N',np.nan)
        name_basics['deathYear']=name_basics['deathYear'].replace(r'\N',np.nan)
        name_basics.to_sql("crew_basics_load",engine,if_exists="append",index=False)
        print("crew_basics_load inserted!")
        del name_basics
        gc.collect()


In [ ]:
#create table crew_basics
sqlcmd='''
delete from crew_basics;
insert into crew_basics
select nconst, "primaryName", cast("birthYear"||'-01-01' as date), cast("deathYear"||'-01-01' as date) from crew_basics_load;
'''
connn.execute(sqlcmd)

In [ ]:
#insert data crew
with pd.read_csv("title.crew.tsv",sep='\t', chunksize=chunksize,error_bad_lines=False) as reader:
    for chunk in reader:
        title_crew=chunk
        title_crew['directors']=title_crew['directors'].replace(r'\N',np.nan)
        title_crew['writers']=title_crew['writers'].replace(r'\N',np.nan)
        title_crew.to_sql("title_crew_load",engine,if_exists="append",index=False)
        print("title_crew_load inserted!")  
        del title_crew
        gc.collect()

In [ ]:
#create table title_director
sqlcmd='''
delete from title_director; 
INSERT INTO public.title_director(title_id,director_id) 
SELECT tconst, directors 
FROM public.title_crew_load;;
'''
connn.execute(sqlcmd)

In [ ]:
#create table tv_episode
title_episode=pd.read_csv("title.episode.tsv",sep='\t',error_bad_lines=False)        
title_episode.rename(columns={"tconst":"title_id","parentTconst":"parent_title_id","seasonNumber":"seasonnumber","episodeNumber":"episodenumber"},inplace=True)
title_episode['seasonnumber']=title_episode['seasonnumber'].replace(r'\N',0)
title_episode['episodenumber']=title_episode['episodenumber'].replace(r'\N',0)
title_episode.drop_duplicates()
title_episode.insert(0,"id",range(1,1+len(title_episode)))
print(title_episode)
title_episode.to_sql("tv_episode",engine,if_exists="append",index=False)
print("tv_episode inserted!")  
del title_episode
gc.collect()
    


In [ ]:
#insert data principals
with pd.read_csv("title.principals.tsv",sep='\t', chunksize=chunksize,error_bad_lines=False) as reader:
    for chunk in reader:
        title_principals=chunk
        title_principals['job']=title_principals['job'].replace(r'\N',np.nan)
        title_principals['characters']=title_principals['characters'].replace(r'\N',np.nan)
        title_principals.to_sql("title_principals_load",engine,if_exists="append",index=False)
        print("title_principals_load inserted!")
        del title_principals
        gc.collect()

In [ ]:
#create table principals
sqlcmd='''
delete from principals;
insert into principals (title_id, crew_id, category, job) 
SELECT tconst, nconst, category, job 
FROM public.title_principals_load;
'''  
connn.execute(sqlcmd)

In [ ]:
#insert data akas
with pd.read_csv("title.akas.tsv",sep='\t', chunksize=chunksize,error_bad_lines=False) as reader:
    for chunk in reader:
        title_akas=chunk
        title_akas['language']=title_akas['language'].replace(r'\N',np.nan)
        title_akas['types']=title_akas['types'].replace(r'\N',np.nan)
        title_akas['attributes']=title_akas['attributes'].replace(r'\N',np.nan)
        title_akas['isOriginalTitle']=title_akas['isOriginalTitle'].replace(r'\N',np.nan)
        title_akas.to_sql("title_akas_load",engine,if_exists="append",index=False)
        print("title_akas_load inserted!")
        del title_akas
        gc.collect()

In [ ]:
#create table title_akas
sqlcmd='''
delete from title_akas;
INSERT INTO title_akas (title_id, title, region, language, attributes, isoriginaltitle) 
SELECT "titleId", title, region, language, attributes 
,case when "isOriginalTitle"='0' then false when "isOriginalTitle"='1' then true else null end 
FROM public.title_akas_load;
'''
connn.execute(sqlcmd)

In [ ]:
#insert data ratings
title_rating=pd.read_csv("title.ratings.tsv",sep='\t', chunksize=chunksize,error_bad_lines=False) 
print(title_rating)

In [ ]:
#create table profession
sqlcmd='''
DELETE FROM profession; 
insert into profession (profession_type) 
select distinct unnest(string_to_array("primaryProfession",',')) from public.crew_basics_load; 
'''
connn.execute(sqlcmd)

In [ ]:
#insert data basics
with pd.read_csv("title.basics.tsv",sep='\t', chunksize=chunksize,error_bad_lines=False) as reader:
    for chunk in reader:
        title_basics=chunk
        title_basics['startYear']=title_basics['startYear'].replace(r'\N',np.nan)
        title_basics['endYear']=title_basics['endYear'].replace(r'\N',np.nan)
        title_basics['runtimeMinutes']=title_basics['runtimeMinutes'].replace(r'\N',np.nan)
        title_basics['isAdult']=title_basics['isAdult'].replace(r'\N',np.nan)
        title_basics.to_sql("title_basics_load",engine,if_exists="append",index=False)
        print("title_basics_load inserted!")
        del title_basics
        gc.collect()

In [ ]:
#insert data ratings
with pd.read_csv("title.ratings.tsv",sep='\t', chunksize=chunksize,error_bad_lines=False) as reader:
    for chunk in reader:
        title_ratings=chunk
        title_ratings.to_sql("title_ratings_load",engine,if_exists="append",index=False)
        print("title_ratings_load inserted!")
        del title_ratings
        gc.collect()

In [ ]:
#create table title_basic
sqlcmd='''
INSERT INTO public.title_basic(
title_id, titletype, name, isadult, startyear, endyear, averagerating, numvotes) 
select distinct b.tconst,b."titleType",coalesce(b."primaryTitle",'') 
,CASE WHEN b."isAdult" = 1 THEN TRUE ELSE FALSE END  
,cast(b."startYear"||'-01-01' as date), 
cast(b."endYear"||'-01-01' as date), r."averageRating", r."numVotes" 
from title_basics_load b left join title_ratings_load r on b.tconst=r.tconst ; 
'''
connn.execute(sqlcmd)

In [ ]:
#create table type
sqlcmd='''
delete from type; 
insert into type (type) 
SELECT distinct  "types" FROM public.title_akas_LOAD WHERE types is not null; 
'''
connn.execute(sqlcmd)

In [ ]:
#create table title_type
sqlcmd='''
DELETE FROM title_type; 
INSERT INTO  public.title_type (uid,type_id) 
select ta.uid,t.type_id from public.type t inner join title_akas_LOAD tal 
on t.type=tal.types inner join title_akas ta on ta.title_id=tal."titleId"; 
'''
connn.execute(sqlcmd)

In [ ]:
#create table genre
sqlcmd='''
delete from genres;
insert into genres (genre_type)
SELECT distinct  "genres" FROM public.title_basics_LOAD WHERE genres is not null; 
'''
connn.execute(sqlcmd)

In [ ]:
#create table title_genre
sqlcmd='''
DELETE FROM title_genre;
INSERT INTO  public.title_genre (genre_id,title_id)
select g.genre_id,tb.title_id from public.genres g inner join title_basics_LOAD tbl
on g.genre_type=tbl.genres inner join title_basic tb on tb.title_id=tbl."tconst";
'''
connn.execute(sqlcmd)

In [ ]:
#create table crew_profession
sqlcmd='''
DELETE FROM crew_profession; 
INSERT INTO  public.crew_profession(crew_id,profession_id) 
select cb.crew_id,p.profession_id from public.profession p inner join crew_basics_LOAD cbl 
on p.profession_type=cbl.primaryProfession inner join crew_basics cb on cb.crew_id=cbl."nconst" ; 
'''
connn.execute(sqlcmd)

In [ ]:
#create table knownfortitles
sqlcmd='''
DELETE FROM knownfortitles; 
INSERT INTO  public.knownfortitles (crew_id,title_id) 
select cb.crew_id,cbl.knownForTitles 
from public.crew_basics as  cb  
inner join crew_basics_LOAD  as cbl on cb.crew_id=cbl."nconst"; 
'''
connn.execute(sqlcmd)